In [7]:
from nltk import regexp_tokenize
from pyspark.sql.functions.builtin import regexp_extract_all

path = '/home/manh/code/nlp/src/data/sentiments.csv'

In [12]:
from datasets import load_dataset

ds = load_dataset("zeroshot/twitter-financial-news-sentiment")

In [14]:
ds.to_csv("sentiments.csv", index=False)

AttributeError: 'DatasetDict' object has no attribute 'to_csv'

In [3]:
from pyspark.sql import SparkSession

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("ReadCSVExample") \
    .getOrCreate()


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/08 21:06:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
df =  spark.read.csv(path , header = True , inferSchema= True)

In [11]:
df = df.groupby("sentiment").count()

df.show()

+---------+-----+
|sentiment|count|
+---------+-----+
|       -1|    1|
|        1|    1|
|     NULL|    1|
+---------+-----+



In [6]:
ff = df.na.drop(subset=['sentiment'])
ff.show(3)

+--------------------+---------+
|                text|sentiment|
+--------------------+---------+
|Kickers on my wat...|        1|
|user: AAP MOVIE. ...|        1|
|user I'd be afrai...|        1|
+--------------------+---------+
only showing top 3 rows


In [7]:
train , test = ff.randomSplit([0.8 , 0.2] , seed = 42)

In [8]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="sentiment", outputCol="indexed_sentiment")
# indexed = indexer.fit(ff).transform(ff)

In [9]:
from pyspark.ml.feature import RegexTokenizer

tokenizer = RegexTokenizer(inputCol ="text", outputCol="token" ,pattern = r'[! ,\-:;?)("\'\n]+' , toLowercase  = True)


In [10]:
from pyspark.ml.feature import StopWordsRemover

remover = StopWordsRemover(inputCol = 'token' , outputCol = "token_removed")

#print(hello world)


In [11]:
from pyspark.ml.feature import CountVectorizer

vectorizer = CountVectorizer(inputCol="token_removed" , outputCol="features" , minDF=10)


In [12]:
from pyspark.ml.classification import LogisticRegression
logistic = LogisticRegression(featuresCol="features", labelCol="indexed_sentiment")


In [13]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages = [tokenizer ,indexer , remover ,  vectorizer , logistic])

model = pipeline.fit(train)

25/11/04 14:24:59 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [14]:
predictions = model.transform(test)
predictions.show(2)

+--------------------+---------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+----------+
|                text|sentiment|               token|indexed_sentiment|       token_removed|            features|       rawPrediction|         probability|prediction|
+--------------------+---------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+----------+
|  ISG An update t...|       -1|[isg, an, update,...|              1.0|[isg, update, feb...|(804,[59,124,201,...|[-4.2542562049815...|[0.01400473307589...|       1.0|
|  The rodeo clown...|       -1|[the, rodeo, clow...|              1.0|[rodeo, clown, se...|(804,[33,141,224,...|[-3.7505890025280...|[0.02296415086506...|       1.0|
+--------------------+---------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+----------

In [15]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(
    labelCol = "indexed_sentiment",
    predictionCol = "prediction",
    metricName = "accuracy"
)

accuracy = evaluator.evaluate(predictions)
accuracy = accuracy * 100
print(accuracy)


77.27682596934174
